In [1]:
import torch
from gector.gec_model import GecBERTModel
from gector.seq2labels_model import Seq2Labels
from allennlp.nn import util

/home/lihao/hdd/anaconda3/envs/gector/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/lihao/hdd/anaconda3/envs/gector/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'
/home/lihao/hdd/anaconda3/envs/gector/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional informa

In [22]:
roberta_gec = GecBERTModel(vocab_path='data/output_vocabulary/', model_paths=['models/roberta_1_gectorv2.th'], is_ensemble=True)
# xlnet_gec = GecBERTModel(vocab_path='data/output_vocabulary/', model_paths=['models/xlnet_0_gectorv2.th'], is_ensemble=True)

In [23]:
input = ['I wants play video games with you'.split(), 
'You know I loves you'.split()]
p, i, e = roberta_gec.predict(roberta_gec.preprocess(input))
roberta_gec.postprocess_batch(input, p, i, e)
# p, i, e = xlnet_gec.predict(xlnet_gec.preprocess(input))
# xlnet_gec.postprocess_batch(input, p, i, e)

bathc: {'tokens': {'bert': tensor([[50265,    38,  1072,   310,   569,   426,    19,    47],
        [50265,   370,   216,    38,  6138,    47,     0,     0]],
       device='cuda:0'), 'bert-offsets': tensor([[0, 1, 2, 3, 4, 5, 6, 7],
        [0, 1, 2, 3, 4, 5, 0, 0]], device='cuda:0'), 'mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0]], device='cuda:0')}}
tensor([[0.9996, 0.9988, 0.9579, 0.9991, 0.9997, 0.9994, 0.9998, 0.4908],
        [0.9984, 0.9980, 0.9877, 0.9990, 0.9167, 0.5697, 0.9984, 0.9984]],
       device='cuda:0') tensor([[ 0,  0, 19,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0, 19,  0,  0,  0]], device='cuda:0') tensor([0.9844, 0.9355], device='cuda:0')


[['I', 'want', 'play', 'video', 'games', 'with', 'you'],
 ['You', 'know', 'I', 'love', 'you']]

In [28]:
class gec(torch.nn.Module):
    def __init__(self, model, model_) -> None:
        super().__init__()
        
        self.model = model
        self.model_ = model_
        
    def forward(self, bert, bert_offsets, mask):
        token = {
            'bert': bert,
            'bert-offsets': bert_offsets,
            'mask': mask
        }
        outputs = self.model.forward(token)
        return self.model_._convert([outputs])

model = Seq2Labels(vocab=roberta_gec.vocab,
                               text_field_embedder=roberta_gec._get_embbeder('roberta-base', 1),
                               confidence=0,
                               del_confidence=0,
                               )
model.load_state_dict(torch.load('models/roberta_1_gectorv2.th'), strict=False)
roberta_gec_onnx = gec(model, roberta_gec).to('cuda:0')

In [29]:
tokens = util.move_to_device(roberta_gec.preprocess(input)[0].as_tensor_dict(), 0)['tokens']
# tokens = util.move_to_device(xlnet_gec.preprocess(input)[0].as_tensor_dict(), 0)['tokens']
print(tokens)

{'bert': tensor([[50265,    38,  1072,   310,   569,   426,    19,    47],
        [50265,   370,   216,    38,  6138,    47,     0,     0]],
       device='cuda:0'), 'bert-offsets': tensor([[0, 1, 2, 3, 4, 5, 6, 7],
        [0, 1, 2, 3, 4, 5, 0, 0]], device='cuda:0'), 'mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0]], device='cuda:0')}


In [30]:
roberta_gec_onnx(tokens['bert'], tokens['bert-offsets'], tokens['mask'])

(tensor([[0.9996, 0.9988, 0.9579, 0.9991, 0.9997, 0.9994, 0.9998, 0.4908],
         [0.9984, 0.9980, 0.9877, 0.9990, 0.9167, 0.5697, 0.9984, 0.9984]],
        device='cuda:0', grad_fn=<MaxBackward0>),
 tensor([[ 0,  0, 19,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0, 19,  0,  0,  0]], device='cuda:0'),
 tensor([0.9844, 0.9355], device='cuda:0', grad_fn=<AddBackward0>))

In [31]:
input_names = ['bert', 'bert-offsets', 'mask']
output_names = ['probs', 'idx', 'error_probs']
dynamic_axes = {
    'bert': {0: 'batch_0', 1: 'sequence_0'},
    'bert-offsets': {0: 'batch_1', 1: 'sequence_1'},
    'mask': {0: 'batch_2', 1: 'sequence_2'},
    'probs': {0: 'batch_3', 1: 'sequence_3'},
    'idx': {0: 'batch_4', 1: 'sequence_4'},
    'error_probs': {0: 'batch_5', 1: 'sequence_5'}
}

torch.onnx.export(roberta_gec_onnx, (tokens['bert'], tokens['bert-offsets'], tokens['mask']), "onnx/roberta-base-new.onnx",
                    input_names=input_names, output_names=output_names, dynamic_axes=dynamic_axes, 
                    opset_version=12)

/home/newdisk/lihao/gector/gector/bert_token_embedder.py:128: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if needs_split:
/home/newdisk/lihao/gector/gector/bert_token_embedder.py:152: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if needs_split:
/home/lihao/hdd/anaconda3/envs/gector/lib/python3.7/site-packages/torch/onnx/symbolic_opset9.py:2819: UserWarning: Exporting aten::index operator of advanced indexing in opset 12 is achieved by combination of multiple ONNX operators, including Reshape, Transpose, Concat, and Gather. If indices include negativ

In [32]:
import onnxruntime
import numpy as np

bert = tokens['bert'].cpu().numpy()
bert_offsets = tokens['bert-offsets'].cpu().numpy()
mask = tokens['mask'].cpu().numpy()

x_onnx = onnxruntime.InferenceSession('onnx/roberta-base-new.onnx', providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])

In [33]:
x_onnx.run(
    None,
    input_feed={
        'bert': bert,
        'bert-offsets': bert_offsets,
        'mask': mask
    }
)

[array([[0.9996369 , 0.99877375, 0.9580072 , 0.99911696, 0.9997173 ,
         0.999388  , 0.9998331 , 0.48995388],
        [0.9983884 , 0.99800795, 0.9877237 , 0.9989674 , 0.9165534 ,
         0.5708694 , 0.9983884 , 0.99838805]], dtype=float32),
 array([[ 0,  0, 19,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0, 19,  0,  0,  0]], dtype=int64),
 array([0.9843983, 0.9354141], dtype=float32)]